In [ ]:
from datetime import timedelta, datetime
import glob
import json
import numpy as np
import os
import pandas as pd
import lib_search_dispersion
import pylab as plt
%matplotlib inline

In [ ]:
dfs = []
for file in glob.glob('data/reverse_search_F*.csv'):
    df = pd.read_csv(file, parse_dates=['start_time', 'end_time'])
    df['sat'] = file.split('.')[0].split('_')[-1]
    dfs.append(df)
df_events = pd.concat(dfs)
df_events.head()

In [ ]:
#df_events = df_events[(df_events.start_time==pd.Timestamp('2015-02-02T09:19:58+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-02-24T22:38:19+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-05-19T21:24:45+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-02-21T15:43:43+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-03-07T04:58:20+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-05-12T11:30:52+00:00'))]

#df_events = df_events[(df_events.start_time==pd.Timestamp('2015-09-08T10:10:07+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-12-24T05:17:23+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-12-24T16:13:23+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-08-29T09:07:18+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-09-11T11:35:38+00:00')) |
#                      (df_events.start_time==pd.Timestamp('2015-11-29T20:51:09+00:00'))
#                     ]

df_events = df_events[
    (df_events.start_time==pd.Timestamp('2015-08-29T09:07:18+00:00')) |
    (df_events.start_time==pd.Timestamp('2015-12-24T16:13:23+00:00'))
]

df_events

In [ ]:
!ls data/training_data_5y/omni/

In [ ]:
case_file = json.load(open('case_files/training_data_5y_F16.json'))
omniweb_fh = lib_search_dispersion.read_omniweb_files(['data/training_data_5y/omni/omni_hro_1min_20160501_v01.cdf'])

In [ ]:
os.makedirs('plots/reverse_search/omni', exist_ok=True)
!rm plots/reverse_search/omni/*.png

In [ ]:
def mk_plot(row, omniweb_fh):
    
    t0 = row['start_time'] - timedelta(minutes=15)
    t1 = row['end_time'] + timedelta(minutes=15)
    
    i = omniweb_fh['t'].searchsorted(t0)
    j = omniweb_fh['t'].searchsorted(t1)

    
    
    mask = np.zeros(omniweb_fh['Bx'].shape, dtype=bool)
    
    for key in ['Bx', 'By', 'Bz', 'n']:
        omniweb_fh[key][omniweb_fh[key] >= 999] = np.nan
    
    
    plt.figure(figsize=(8, 8))
    
    #plt.suptitle(f'Event Detection\n{row["start_time"].isoformat()} - {row["end_time"].isoformat()} ({row["sat"]})')
    plt.subplot(411)
    plt.title('IMF Field Around Event from OMNI')
    plt.plot(omniweb_fh['t'][i:j], omniweb_fh['Bx'][i:j], 'b-', label='Bx')
    plt.plot(omniweb_fh['t'][i:j], omniweb_fh['By'][i:j], 'g-', label='By')
    plt.plot(omniweb_fh['t'][i:j], omniweb_fh['Bz'][i:j], 'r-', label='Bz')
    plt.legend(ncol=3)
    #plt.ylim([-15, 15])
    plt.grid(linestyle='dashed', color='#ddd')
    plt.ylabel('$\\vec{B}$ (nT)')
    plt.fill_between([row["start_time"], row["end_time"]],
                     [plt.ylim()[0]]*2, [plt.ylim()[1]]*2,
                     color='gray', alpha=0.2)
    plt.xlim(t0, t1)
    
    plt.subplot(412)
    plt.plot(omniweb_fh['t'][i:j], omniweb_fh['Bz'][i:j], 'r-', label='Bz')
    plt.legend(ncol=1)
    #plt.ylim([-35, 35])
    plt.grid(linestyle='dashed', color='#ddd')
    plt.ylabel('Bz (nT)')
    plt.fill_between([row["start_time"], row["end_time"]],
                     [plt.ylim()[0]]*2, [plt.ylim()[1]]*2,
                     color='gray', alpha=0.2)
    plt.xlim(t0, t1)
    
    plt.subplot(413)
    plt.plot(omniweb_fh['t'][i:j],
             np.sqrt(omniweb_fh['Bx'][i:j]**2 + omniweb_fh['By'][i:j]**2 + omniweb_fh['Bz'][i:j]**2),
              'k-', label='|B|')
    plt.legend()
    plt.ylim([0, 15])
    plt.grid(linestyle='dashed', color='#ddd')
    plt.ylabel('|B| (nT)')
    plt.fill_between([row["start_time"], row["end_time"]],
                     [plt.ylim()[0]]*2, [plt.ylim()[1]]*2,
                     color='gray', alpha=0.2)
    plt.xlim(t0, t1)
    
    plt.subplot(414)
    plt.plot(omniweb_fh['t'][i:j], omniweb_fh['n'][i:j], color='orange', label='n')
    plt.legend()
    #plt.ylim([0, 25])
    plt.grid(linestyle='dashed', color='#ddd')
    plt.ylabel('n ($cm^{-3}$)')
    plt.fill_between([row["start_time"], row["end_time"]],
                     [plt.ylim()[0]]*2, [plt.ylim()[1]]*2,
                     color='gray', alpha=0.2)
    plt.xlim(t0, t1)
    plt.tight_layout()

In [ ]:
!mkdir -p plots/paper_sci_plots/omni

In [ ]:
import progressbar

for k in omniweb_fh.keys():
    if k == 't': continue
    mask = omniweb_fh[k] > 800
    omniweb_fh[k][mask] = np.nan

bar = progressbar.ProgressBar()
    
for _, row in bar(list(df_events.iterrows())):
    mk_plot(row, omniweb_fh)
    file_name = f'plots/paper_sci_plots/omni/omni_{row.start_time.isoformat()}_{row.end_time.isoformat()}.png'
    plt.savefig(file_name)
    #print(file_name)

    plt.close()

In [ ]:
import pytz

mk_plot(
    row=dict(
        start_time=datetime(2016,5,21, 6,22, 19, tzinfo=pytz.utc), 
        end_time=datetime(2016, 5 ,21, 6,23,31, tzinfo=pytz.utc),
        sat=16,
    ),
    omniweb_fh=omniweb_fh
)

In [ ]:
omniweb_fh['n'].max()